In [ ]:
"""
To make sure the fusion decomposition works
up to the part where we would stitch the sublimbs together into one limb


"""

In [1]:
from os import sys
sys.path.append("/meshAfterParty/")

In [2]:
import skeleton_utils as sk
import soma_extraction_utils as sm
import trimesh_utils as tu
import trimesh
import numpy_utils as nu
import numpy as np
from importlib import reload
import networkx as nx
import time
import compartment_utils as cu
import networkx_utils as xu
import matplotlib_utils as mu

#importing at the bottom so don't get any conflicts
import itertools
from tqdm_utils import tqdm

#for meshparty preprocessing
import meshparty_skeletonize as m_sk
import general_utils as gu
import compartment_utils as cu
from meshparty import trimesh_io
from copy import deepcopy


from neuron_utils import *


# Loading the Neuron

In [3]:
import trimesh_utils as tu
curent_neuron = tu.load_mesh_no_processing("/notebooks/test_neurons/Segmentation_2/meshparty/864691135548568516_single_soma_inhib_axon_cloud.off")

In [4]:
segment_id = 864691135548568516
description = "single_soma_inhib_axon_cloud"

# Setting the Arguments that would be present inside a preprocessing function call

In [5]:
#predefined arguments for the Neuron constructor

decomposition_type="meshafterparty"
mesh_correspondence="meshparty" #meshafterparty_adaptive
distance_by_mesh_center=True #how the distance is calculated for mesh correspondence
meshparty_segment_size = 100
meshparty_n_surface_downsampling = 2
meshparty_adaptive_correspondence_after_creation=False
suppress_preprocessing_print=True
computed_attribute_dict=None
somas = None
branch_skeleton_data=None
combine_close_skeleton_nodes = True
combine_close_skeleton_nodes_threshold=700
ignore_warnings=True
suppress_output=False
calculate_spines=True
widths_to_calculate=["no_spine_median_mesh_center",
                "no_spine_mean_mesh_center"]
fill_hole_size=2000

In [6]:
#arguments for the preprocess neuron
mesh = curent_neuron
segment_id=segment_id
description=description

sig_th_initial_split=15 #for significant splitting meshes in the intial mesh split
limb_threshold = 2000 #the mesh faces threshold for a mesh to be qualified as a limb (otherwise too small)
filter_end_node_length=4001 #used in cleaning the skeleton during skeletonizations
return_no_somas = False

decomposition_type=decomposition_type
mesh_correspondence=mesh_correspondence
distance_by_mesh_center=distance_by_mesh_center
meshparty_segment_size =meshparty_segment_size
meshparty_n_surface_downsampling = meshparty_n_surface_downsampling
somas=somas
branch_skeleton_data=branch_skeleton_data
combine_close_skeleton_nodes = combine_close_skeleton_nodes
combine_close_skeleton_nodes_threshold=combine_close_skeleton_nodes_threshold

# Setting up the metadata

In [7]:
whole_processing_tiempo = time.time()


"""
Purpose: To process the mesh into a format that can be loaded into the neuron class
and used for higher order processing (how to visualize is included)

"""
if description is None:
    description = "no_description"
if segment_id is None:
    #pick a random segment id
    segment_id = np.random.randint(100000000)
    print(f"picking a random 7 digit segment id: {segment_id}")
    description += "_random_id"


if mesh is None:
    if current_mesh_file is None:
        raise Exception("No mesh or mesh_file file were given")
    else:
        current_neuron = trimesh.load_mesh(current_mesh_file)
else:
    current_neuron = mesh

# ************************ Phase A: Soma and Limb Identification ********************************

In [8]:
# --- 1) Doing the soma detection
if somas is None:
    soma_mesh_list,run_time,total_soma_list_sdf = sm.extract_soma_center(segment_id,
                                             current_neuron.vertices,
                                             current_neuron.faces)
else:
    soma_mesh_list,run_time,total_soma_list_sdf = somas

# geting the soma centers
if len(soma_mesh_list) <= 0:
    print(f"**** No Somas Found for Mesh {segment_id} so just one mesh")
    soma_mesh_list_centers = []
    if return_no_somas:
        return_value= soma_mesh_list_centers
    raise Exception("Processing of No Somas is not yet implemented yet")
else:
    #compute the soma centers
    print(f"Soma List = {soma_mesh_list}")

    soma_mesh_list_centers = sm.find_soma_centroids(soma_mesh_list)
    print(f"soma_mesh_list_centers = {soma_mesh_list_centers}")

Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 15000.0 
large_mesh_threshold_inner = 10000.0 
soma_size_threshold = 937.5 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
xvfb-run -n 5951 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Decomp_Fusion/864691135548568516/neuron_864691135548568516.off -o /notebooks/Platinum_Decomp_Fusion/864691135548568516/neuron_864691135548568516_decimated.off -s /notebooks/Platinum_Decomp_Fusion/864691135548568516/decimation_meshlab_25768543.mls
Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(191269, 3), faces.shape=(379018, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(191269, 3), faces.shape=(379018, 3))>
pre_largest_mesh_path = /notebooks/Platinum_Decomp_Fusion/864691135548568516/neuron_864691135548568516_decimated_largest_piece.off
xvfb-run -n 3601 -s "-screen 0 800x600x24" meshlabserver $@  -i /noteb

In [9]:
#--- 2) getting the soma submeshes that are connected to each soma and identifiying those that aren't (and eliminating any mesh pieces inside the soma)

main_mesh_total = current_neuron


#finding the mesh pieces that contain the soma
#splitting the current neuron into distinct pieces
split_meshes = tu.split_significant_pieces(
                            main_mesh_total,
                            significance_threshold=sig_th_initial_split,
                            print_flag=False)

print(f"# total split meshes = {len(split_meshes)}")


#returns the index of the split_meshes index that contains each soma    
containing_mesh_indices = sm.find_soma_centroid_containing_meshes(soma_mesh_list,
                                        split_meshes)

# filtering away any of the inside floating pieces: 
non_soma_touching_meshes = [m for i,m in enumerate(split_meshes)
                 if i not in list(containing_mesh_indices.values())]


#Adding the step that will filter away any pieces that are inside the soma
if len(non_soma_touching_meshes) > 0 and len(soma_mesh_list) > 0:
    """
    *** want to save these pieces that are inside of the soma***
    """

    non_soma_touching_meshes,inside_pieces = sm.filter_away_inside_soma_pieces(soma_mesh_list,non_soma_touching_meshes,
                                    significance_threshold=sig_th_initial_split,
                                    return_inside_pieces = True)                                                      


split_meshes # the meshes of the original mesh
containing_mesh_indices #the mapping of each soma centroid to the correct split mesh
soma_containing_meshes = sm.grouping_containing_mesh_indices(containing_mesh_indices)

soma_touching_meshes = [split_meshes[k] for k in soma_containing_meshes.keys()]


#     print(f"# of non soma touching seperate meshes = {len(non_soma_touching_meshes)}")
#     print(f"# of inside pieces = {len(inside_pieces)}")
print(f"# of soma containing seperate meshes = {len(soma_touching_meshes)}")
print(f"meshes with somas = {soma_containing_meshes}")






# total split meshes = 35
There were 34 pieces found after size threshold
# of soma containing seperate meshes = 1
meshes with somas = {0: [0]}


In [10]:
tu = reload(tu)
#--- 3)  Soma Extraction was great (but it wasn't the original soma faces), so now need to get the original soma faces and the original non-soma faces of original pieces

#     sk.graph_skeleton_and_mesh(other_meshes=[soma_meshes])

"""
for each soma touching mesh get the following:
1) original soma meshes
2) significant mesh pieces touching these somas
3) The soma connectivity to each of the significant mesh pieces
-- later will just translate the 


Process: 

1) Final all soma faces (through soma extraction and then soma original faces function)
2) Subtact all soma faces from original mesh
3) Find all significant mesh pieces
4) Backtrack significant mesh pieces to orignal mesh and find connectivity of each to all
   the available somas
Conclusion: Will have connectivity map


"""

soma_touching_mesh_data = dict()

for z,(mesh_idx, soma_idxes) in enumerate(soma_containing_meshes.items()):
    soma_touching_mesh_data[z] = dict()
    print(f"\n\n----Working on soma-containing mesh piece {z}----")

    #1) Final all soma faces (through soma extraction and then soma original faces function)
    current_mesh = split_meshes[mesh_idx]

    current_soma_mesh_list = [soma_mesh_list[k] for k in soma_idxes]

    current_time = time.time()
    mesh_pieces_without_soma = sm.subtract_soma(current_soma_mesh_list,current_mesh,
                                                significance_threshold=250)
    print(f"Total time for Subtract Soam = {time.time() - current_time}")
    current_time = time.time()

    mesh_pieces_without_soma_stacked = tu.combine_meshes(mesh_pieces_without_soma)

    # find the original soma faces of mesh
    soma_faces = tu.original_mesh_faces_map(current_mesh,mesh_pieces_without_soma_stacked,matching=False)
    print(f"Total time for Original_mesh_faces_map for mesh_pieces without soma= {time.time() - current_time}")
    current_time = time.time()
    soma_meshes = current_mesh.submesh([soma_faces],append=True,repair=False)

    # finding the non-soma original faces
    non_soma_faces = tu.original_mesh_faces_map(current_mesh,soma_meshes,matching=False)
    non_soma_stacked_mesh = current_mesh.submesh([non_soma_faces],append=True,repair=False)

    print(f"Total time for Original_mesh_faces_map for somas= {time.time() - current_time}")
    current_time = time.time()

    # 3) Find all significant mesh pieces
    sig_non_soma_pieces,insignificant_limbs = tu.split_significant_pieces(non_soma_stacked_mesh,significance_threshold=limb_threshold,
                                                     return_insignificant_pieces=True)

    print(f"Total time for sig_non_soma_pieces= {time.time() - current_time}")
    current_time = time.time()

    soma_touching_mesh_data[z]["branch_meshes"] = sig_non_soma_pieces

    #4) Backtrack significant mesh pieces to orignal mesh and find connectivity of each to all the available somas
    # get all the seperate mesh faces

    #How to seperate the mesh faces
    seperate_soma_meshes,soma_face_components = tu.split(soma_meshes,only_watertight=False)
    #take the top largest ones depending how many were originally in the soma list
    seperate_soma_meshes = seperate_soma_meshes[:len(soma_mesh_list)]
    soma_face_components = soma_face_components[:len(soma_mesh_list)]

    soma_touching_mesh_data[z]["soma_meshes"] = seperate_soma_meshes

    print(f"Total time for split= {time.time() - current_time}")
    current_time = time.time()



    soma_to_piece_connectivity = dict()
    soma_to_piece_touching_vertices = dict()
    soma_to_piece_touching_vertices_idx = dict()
    limb_root_nodes = dict()
    
    m_vert_graph = tu.mesh_vertex_graph(current_mesh)
    
    for i,curr_soma in enumerate(seperate_soma_meshes):
        (connected_mesh_pieces,
         connected_mesh_pieces_vertices,
         connected_mesh_pieces_vertices_idx) = tu.mesh_pieces_connectivity(
                        main_mesh=current_mesh,
                        central_piece=curr_soma,
                        periphery_pieces = sig_non_soma_pieces,
                        return_vertices = True,
                        return_vertices_idx=True)
        #print(f"soma {i}: connected_mesh_pieces = {connected_mesh_pieces}")
        soma_to_piece_connectivity[i] = connected_mesh_pieces

        soma_to_piece_touching_vertices[i] = dict()
        for piece_index,piece_idx in enumerate(connected_mesh_pieces):
            limb_root_nodes[piece_idx] = connected_mesh_pieces_vertices[piece_index][0]
            
            """ Old way of finding vertex connected components on a mesh without trimesh function
            #find the number of touching groups and save those 
            soma_touching_graph = m_vert_graph.subgraph(connected_mesh_pieces_vertices_idx[piece_index])
            soma_con_comp = [current_mesh.vertices[np.array(list(k)).astype("int")] for k in list(nx.connected_components(soma_touching_graph))]
            soma_to_piece_touching_vertices[i][piece_idx] = soma_con_comp
            """
            
            soma_to_piece_touching_vertices[i][piece_idx] = tu.split_vertex_list_into_connected_components(
                                                vertex_indices_list=connected_mesh_pieces_vertices_idx[piece_index],
                                                mesh=current_mesh, 
                                                vertex_graph=m_vert_graph, 
                                                return_coordinates=True
                                               )
            
            
            
            

#         border_debug = False
#         if border_debug:
#             print(f"soma_to_piece_connectivity = {soma_to_piece_connectivity}")
#             print(f"soma_to_piece_touching_vertices = {soma_to_piece_touching_vertices}")


    print(f"Total time for mesh_pieces_connectivity= {time.time() - current_time}")

    soma_touching_mesh_data[z]["soma_to_piece_connectivity"] = soma_to_piece_connectivity

print(f"# of insignificant_limbs = {len(insignificant_limbs)} with trimesh : {insignificant_limbs}")



# Lets have an alert if there was more than one soma disconnected meshes
if len(soma_touching_mesh_data.keys()) > 1:
    raise Exception("More than 1 disconnected meshes that contain somas")

current_mesh_data = soma_touching_mesh_data
soma_containing_idx = 0

#doing inversion of the connectivity and touching vertices
piece_to_soma_touching_vertices = gu.flip_key_orders_for_dict(soma_to_piece_touching_vertices)


# ****Soma Touching mesh Data has the branches and the connectivity (So this is where you end up skipping if you don't have somas)***



----Working on soma-containing mesh piece 0----

inside Soma subtraction
mesh pieces in subtact soma BEFORE the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(442924, 3), faces.shape=(883241, 3))>, <trimesh.Trimesh(vertices.shape=(108008, 3), faces.shape=(215445, 3))>, <trimesh.Trimesh(vertices.shape=(75957, 3), faces.shape=(151431, 3))>, <trimesh.Trimesh(vertices.shape=(54061, 3), faces.shape=(107833, 3))>, <trimesh.Trimesh(vertices.shape=(42023, 3), faces.shape=(83820, 3))>, <trimesh.Trimesh(vertices.shape=(17933, 3), faces.shape=(35808, 3))>]
There were 6 pieces found after size threshold
mesh pieces in subtact soma AFTER the filtering inside pieces = [<trimesh.Trimesh(vertices.shape=(442924, 3), faces.shape=(883241, 3))>, <trimesh.Trimesh(vertices.shape=(108008, 3), faces.shape=(215445, 3))>, <trimesh.Trimesh(vertices.shape=(75957, 3), faces.shape=(151431, 3))>, <trimesh.Trimesh(vertices.shape=(54061, 3), faces.shape=(107833, 3))>, <trimesh.Trimesh(vertices.shape=(42

# Process that will start for each limb

In [11]:
import preprocessing_vp2 as pre
from copy import deepcopy

In [142]:
sk = reload(sk)
tu = reload(tu)
cu = reload(cu)
m_sk = reload(m_sk)

proper_limb_mesh_correspondence_MAP = dict()
proper_limb_mesh_correspondence_MP = dict()


meshparty_segment_size = 200

proper_time = time.time()
print_fusion_steps=True

#parameters for the stitching:

move_MAP_stitch_to_end_or_branch = True
distance_to_move_point_threshold=500

for curr_limb_idx,limb_mesh_mparty in enumerate(current_mesh_data[0]["branch_meshes"]):
    
#     if curr_limb_idx != 5:
#         continue
    
    curr_limb_time = time.time()
    print(f"\n\n----- Working on Proper Limb #{curr_limb_idx} ---------")
    
    
    
    fusion_time = time.time()
    # Doing the first pass of surface skeletonization
    total_border_vertices = dict()
    for l_idx,s_dict in piece_to_soma_touching_vertices.items():
        local_b_verts = []
        for sm_idx,sm_bord_verts in s_dict.items():
            local_b_verts.append(np.vstack(sm_bord_verts))
        total_border_vertices[l_idx] = np.vstack(local_b_verts)

    #will eventually get the current root from soma_to_piece_touching_vertices[i]
    root_curr = total_border_vertices[curr_limb_idx][0]
    if print_fusion_steps:
        print(f"Time for preparing soma vertices and root: {time.time() - fusion_time }")
        fusion_time = time.time()
    
    sk_meshparty_obj = m_sk.skeletonize_mesh_largest_component(limb_mesh_mparty,
                                                            root=root_curr,
                                                              filter_mesh=False)

    if print_fusion_steps:
        print(f"Time for 1st pass MP skeletonization: {time.time() - fusion_time }")
        fusion_time = time.time()
        
    (segment_branches, #skeleton branches
    divided_submeshes, divided_submeshes_idx, #mesh correspondence (mesh and indices)
    segment_widths_median) = m_sk.skeleton_obj_to_branches(sk_meshparty_obj,
                                                          mesh = limb_mesh_mparty,
                                                          meshparty_segment_size=meshparty_segment_size)
    
    
    if print_fusion_steps:
        print(f"Decomposing first pass: {time.time() - fusion_time }")
        fusion_time = time.time()
    
    
    
    #using the width threshold
    width_threshold = 450
    size_threshold = 1000

    #gettin the branches that should be passed through MAP skeletonization
    pieces_above_threshold = np.where(segment_widths_median>width_threshold)[0]

    #getting the correspondnece info for those MAP qualifying
    width_large = segment_widths_median[pieces_above_threshold]
    sk_large = [segment_branches[k] for k in pieces_above_threshold]
    mesh_large_idx = [divided_submeshes_idx[k] for k in pieces_above_threshold]
    
    

    mesh_pieces_for_MAP = []

    if len(mesh_large_idx) > 0: #will only continue processing if found MAP candidates
        
        print(f"Found len(mesh_large_idx) MAP candidates: {[len(k) for k in mesh_large_idx]}")
        
        #finds the connectivity edges of all the MAP candidates
        mesh_large_connectivity = tu.mesh_list_connectivity(meshes = mesh_large_idx,
                                main_mesh = limb_mesh_mparty,
                                print_flag = False)
        if print_fusion_steps:
            print(f"mesh_large_connectivity: {time.time() - fusion_time }")
            fusion_time = time.time()
        """
        --------------- Grouping MAP candidates ----------------
        Purpose: Will see what mesh pieces should be grouped together
        to pass through CGAL skeletonization


        Pseudocode: 
        1) build a networkx graph with all nodes for mesh_large_idx indexes
        2) Add the edges
        3) Find the connected components
        4) Find sizes of connected components
        5) For all those connected components that are of a large enough size, 
        add the mesh branches and skeletons to the final list


        """
        G = nx.Graph()
        G.add_nodes_from(np.arange(len(mesh_large_idx)))
        G.add_edges_from(mesh_large_connectivity)
        conn_comp = list(nx.connected_components(G))

        filtered_pieces = []

        sk_large_size_filt = []
        mesh_large_idx_size_filt = []
        width_large_size_filt = []

        for cc in conn_comp:
            total_cc_size = np.sum([len(mesh_large_idx[k]) for k in cc])
            if total_cc_size>size_threshold:
                #print(f"cc ({cc}) passed the size threshold because size was {total_cc_size}")
                filtered_pieces.append(pieces_above_threshold[list(cc)])
                
        if print_fusion_steps:
            print(f"Finding MAP candidates connected components: {time.time() - fusion_time }")
            fusion_time = time.time()

        #filtered_pieces: will have the indexes of all the branch candidates that should  be 
        #grouped together and passed through MAP skeletonization

        if len(filtered_pieces) > 0:
            print(f"len(filtered_pieces) = {len(filtered_pieces)}")
            #all the pieces that will require MAP mesh correspondence and skeletonization
            #(already organized into their components)
            mesh_pieces_for_MAP = [limb_mesh_mparty.submesh([np.concatenate(divided_submeshes_idx[k])],append=True,repair=False) for k in filtered_pieces]
            mesh_pieces_for_MAP_face_idx = [np.concatenate(divided_submeshes_idx[k]) for k in filtered_pieces]
            
            pieces_idx_MP = np.setdiff1d(np.arange(len(divided_submeshes_idx)),np.concatenate(filtered_pieces))

            """
            Old Way: Finding connectivity of pieces through
            mesh_idx_MP = [divided_submeshes_idx[k] for k in pieces_idx_MP]

            mesh_large_connectivity_MP = tu.mesh_list_connectivity(meshes = mesh_idx_MP,
                                    main_mesh = limb_mesh_mparty,
                                    print_flag = False)

            New Way: going to use skeleton connectivity to determine
            connectivity of pieces

            Pseudocode: 
            1)

            """

            skeleton_MP = [segment_branches[k] for k in pieces_idx_MP]
            skeleton_connectivity_MP = sk.skeleton_list_connectivity(
                                            skeletons=skeleton_MP
                                            )
            if print_fusion_steps:
                print(f"skeleton_connectivity_MP : {time.time() - fusion_time }")
                fusion_time = time.time()

            G = nx.Graph()
            G.add_nodes_from(np.arange(len(skeleton_MP)))
            G.add_edges_from(skeleton_connectivity_MP)
            sublimbs_MP = list(nx.connected_components(G))
            sublimbs_MP_orig_idx = [pieces_idx_MP[list(k)] for k in sublimbs_MP]


            #concatenate into sublimbs the skeletons and meshes
            sublimb_mesh_idx_branches_MP = [divided_submeshes_idx[k] for k in sublimbs_MP_orig_idx]
            sublimb_mesh_branches_MP = [[limb_mesh_mparty.submesh([ki],append=True,repair=False)
                                        for ki in k] for k in sublimb_mesh_idx_branches_MP]
            sublimb_meshes_MP = [limb_mesh_mparty.submesh([np.concatenate(k)],append=True,repair=False)
                                                         for k in sublimb_mesh_idx_branches_MP]
            sublimb_meshes_MP_face_idx = [np.concatenate(k)
                                                         for k in sublimb_mesh_idx_branches_MP]
            sublimb_skeleton_branches = [segment_branches[k] for k in sublimbs_MP_orig_idx]
            widths_MP = [segment_widths_median[k] for k in sublimbs_MP_orig_idx]
            
            if print_fusion_steps:
                print(f"Grouping MP Sublimbs by Graph: {time.time() - fusion_time }")
                fusion_time = time.time()

    # else: #if no pieces were determine to need MAP processing
    #     print("No MAP processing needed: just returning the Meshparty skeletonization and mesh correspondence")
    #     raise Exception("Returning MP correspondence")


    # nviz.plot_objects(main_mesh=tu.combine_meshes([limb_mesh_mparty,current_neuron["S0"].mesh]),
    #                   main_mesh_color="green",
    #     skeletons=sk_large_size_filt,
    #      meshes=[limb_mesh_mparty.submesh([k],append=True) for k in mesh_large_idx_size_filt],
    #       meshes_colors="red")
    
    
    
    
    
    
    
    
    
    
    #if no sublimbs need to be decomposed with MAP then just reassign all of the previous MP processing to the sublimb_MPs
    if len(mesh_pieces_for_MAP) == 0:
        sublimb_meshes_MP = [limb_mesh_mparty] #trimesh pieces that have already been passed through MP skeletonization (may not need)
        # -- the decomposition information ---
        sublimb_mesh_branches_MP = [divided_submeshes] #the mesh branches for all the disconnected sublimbs
        sublimb_mesh_idx_branches_MP = [divided_submeshes_idx] #The mesh branches idx that have already passed through MP skeletonization
        sublimb_skeleton_branches = [segment_branches]#the skeleton bnraches for all the sublimbs
        widths_MP = [segment_widths_median] #the mesh branches widths for all the disconnected groups

        MAP_flag = False
    else:
        MAP_flag = True

    

    mesh_pieces_for_MAP #trimesh pieces that should go through CGAL skeletonization
    sublimb_meshes_MP #trimesh pieces that have already been passed through MP skeletonization (may not need)

    # -- the decomposition information ---
    sublimb_mesh_branches_MP #the mesh branches for all the disconnected sublimbs
    sublimb_mesh_idx_branches_MP #The mesh branches idx that have already passed through MP skeletonization
    sublimb_skeleton_branches #the skeleton bnraches for all the sublimbs
    widths_MP #the mesh branches widths for all the disconnected groups
    
    if print_fusion_steps:
        print(f"Divinding into MP and MAP pieces: {time.time() - fusion_time }")
        fusion_time = time.time()

    
    
    # ------------------- At this point have the correct division between MAP and MP ------------------------
    
    # -------------- Doing the MAP decomposition ------------------ #
    global_start_time = time.time()
    endpoints_must_keep = dict()

    perform_cleaning_checks = False

    limb_correspondence_MAP = dict()
    
    for sublimb_idx,(mesh,mesh_idx) in enumerate(zip(mesh_pieces_for_MAP,mesh_pieces_for_MAP_face_idx)):
        print(f"--- Working on MAP piece {sublimb_idx}---")
        mesh_start_time = time.time()
        curr_soma_to_piece_touching_vertices = pre.filter_soma_touching_vertices_dict_by_mesh(
        mesh = mesh,
        curr_piece_to_soma_touching_vertices = piece_to_soma_touching_vertices[curr_limb_idx]
        )
        
        if print_fusion_steps:
            print(f"MAP Filtering Soma Pieces: {time.time() - fusion_time }")
            fusion_time = time.time()

        # ---- 0) Generating the Clean skeletons  -------------------------------------------#
        if not curr_soma_to_piece_touching_vertices is None:
            total_border_vertices = dict([(k,np.vstack(v)) for k,v in curr_soma_to_piece_touching_vertices.items()])

        cleaned_branch,curr_limb_endpoints_must_keep = sk.skeletonize_and_clean_connected_branch_CGAL(
            mesh=mesh,
            curr_soma_to_piece_touching_vertices=curr_soma_to_piece_touching_vertices,
            total_border_vertices=total_border_vertices,
            filter_end_node_length=filter_end_node_length,
            perform_cleaning_checks=perform_cleaning_checks,
            combine_close_skeleton_nodes = combine_close_skeleton_nodes,
            combine_close_skeleton_nodes_threshold=combine_close_skeleton_nodes_threshold)

        if len(cleaned_branch) == 0:
            raise Exception(f"Found a zero length skeleton for limb {z} of trmesh {branch}")
            
        if print_fusion_steps:
            print(f"skeletonize_and_clean_connected_branch_CGAL: {time.time() - fusion_time }")
            fusion_time = time.time()

        # ---- 1) Generating Initial Mesh Correspondence -------------------------------------------#
        start_time = time.time()

        print(f"Working on limb correspondence for #{sublimb_idx} MAP piece")
        local_correspondence = pre.mesh_correspondence_first_pass(mesh=mesh,
                                                             skeleton=cleaned_branch,
                                                             distance_by_mesh_center=distance_by_mesh_center)


        print(f"Total time for decomposition = {time.time() - start_time}")
        if print_fusion_steps:
            print(f"mesh_correspondence_first_pass: {time.time() - fusion_time }")
            fusion_time = time.time()


        #------------- 2) Doing Some checks on the initial corespondence -------- #

        perform_cleaning_checks = True       
        if perform_cleaning_checks:
            pre.check_skeletonization_and_decomp(skeleton=cleaned_branch,
                                            local_correspondence=local_correspondence)

        # -------3) Finishing off the face correspondence so get 1-to-1 correspondence of mesh face to skeletal piece
        local_correspondence_revised = pre.correspondence_1_to_1(mesh=mesh,
                                        local_correspondence=local_correspondence,
                                        curr_limb_endpoints_must_keep=curr_limb_endpoints_must_keep,
                                        curr_soma_to_piece_touching_vertices=curr_soma_to_piece_touching_vertices)
        
        # -------3b) Fixing the mesh indices to correspond to the larger mesh as a whole
        for k,v in local_correspondence_revised.items():
            local_correspondence_revised[k]["branch_face_idx"] = mesh_idx[local_correspondence_revised[k]["branch_face_idx"]]
        
        print(f"Total time for MAP sublimb #{sublimb_idx} mesh processing = {time.time() - mesh_start_time}")
        
        if print_fusion_steps:
            print(f"correspondence_1_to_1: {time.time() - fusion_time }")
            fusion_time = time.time()

        limb_correspondence_MAP[sublimb_idx] = local_correspondence_revised

    print(f"Total time for MAP sublimb processing {time.time() - global_start_time}")

    

    
    
    # ----------------- Part B: Doing the MP Decomposition ---------------------- #
    
    
    
    
    sublimb_meshes_MP #trimesh pieces that have already been passed through MP skeletonization (may not need)
    # -- the decomposition information ---
    sublimb_mesh_branches_MP #the mesh branches for all the disconnected sublimbs
    sublimb_mesh_idx_branches_MP #The mesh branches idx that have already passed through MP skeletonization
    sublimb_skeleton_branches #the skeleton bnraches for all the sublimbs
    widths_MP #the mesh branches widths for all the disconnected groups

    limb_correspondence_MP = dict()

    print("***************Arrived at MESH PIECES FOR MAP AND mp***************")
    for sublimb_idx,mesh in enumerate(sublimb_meshes_MP):
        print(f"---- Working on MP Decomposition #{sublimb_idx} ----")
        mesh_start_time = time.time()

        if len(sublimb_meshes_MP) == 1 and MAP_flag == False:
            print("Using Quicker soma_to_piece_touching_vertices because no MAP and only one sublimb_mesh piece ")
            curr_soma_to_piece_touching_vertices = piece_to_soma_touching_vertices[curr_limb_idx]
        else:
            curr_soma_to_piece_touching_vertices = pre.filter_soma_touching_vertices_dict_by_mesh(
                                                mesh = mesh,
                                                curr_piece_to_soma_touching_vertices = piece_to_soma_touching_vertices[curr_limb_idx]
                                                )
            
        if print_fusion_steps:
            print(f"MP filtering soma verts: {time.time() - fusion_time }")
            fusion_time = time.time()

        #creating all of the sublimb groups
        segment_branches = sublimb_skeleton_branches[sublimb_idx]
        whole_sk_MP = sk.stack_skeletons(segment_branches)
        branch = mesh
        divided_submeshes = sublimb_mesh_branches_MP[sublimb_idx]
        divided_submeshes_idx = sublimb_mesh_idx_branches_MP[sublimb_idx]
        segment_widths_median = widths_MP[sublimb_idx]

        if curr_soma_to_piece_touching_vertices is None:
            print(f"Do Not Need to Fix MP Decomposition {sublimb_idx} so just continuing")

        else:
            print(f"Fixing Possible Soma Extension Branch for Sublimb {sublimb_idx}")

            #If there is some soma touching then need to see if have to fix soma extending pieces
            return_info = sk.create_soma_extending_branches(current_skeleton=whole_sk_MP,
                                      skeleton_mesh=branch,
                                      soma_to_piece_touching_vertices=curr_soma_to_piece_touching_vertices,
                                      return_endpoints_must_keep=True,
                                         return_created_branch_info=True)
            new_sk,endpts,new_branch_info = return_info

            if print_fusion_steps:
                print(f"MP (because soma touching verts) create_soma_extending_branches: {time.time() - fusion_time }")
                fusion_time = time.time()

            no_soma_extension_add = True
            for sm_idx in new_branch_info.keys():
                for b_vert_idx,br_info in enumerate(new_branch_info[sm_idx]):
                    if br_info is None:
                        continue
                    no_soma_extension_add=False


                    #1) Get the newly added branch (and the original vertex which is the first row)
                    br_new,sm_bord_verts = br_info #this will hold the new branch and the border vertices corresponding to it

                    curr_soma_to_piece_touching_vertices_MP = {sm_idx:[sm_bord_verts]}
                    endpoints_must_keep_MP = {sm_idx:[br_new[0][1]]}

                    orig_vertex = br_new[0][0]

                    #2) Find the branches that have that coordinate (could be multiple)
                    match_sk_branches = sk.find_branch_skeleton_with_specific_coordinate(segment_branches,
                        current_coordinate=orig_vertex)

                    #2b) If multiple branches: Pick a winning branch that has the most of the
                    #    soma touching vertices
                    if len(match_sk_branches) > 1:
                        bord_verts_tree = KDTree(sm_bord_verts.reshape(-1,3))
                        winning_branch = match_sk_branches[0]
                        dist,_ = bord_verts_tree.query(divided_submeshes[winning_branch].vertices)
                        winning_branch_n_bord_verts = np.sum(dist == 0)

                        for i in range(1,len(match_sk_branches)):
                            curr_branch = match_sk_branches[i]
                            dist,_ = bord_verts_tree.query(divided_submeshes[winning_branch].vertices)
                            n_bord_verts = np.sum(dist == 0)
                            if n_bord_verts>winning_branch_n_bord_verts:
                                winning_branch_n_bord_verts = n_bord_verts
                                winning_branch = curr_branch
                    elif len(match_sk_branches) == 1:
                        winning_branch = match_sk_branches[0]
                    else:
                        raise Exception("No matching branches found for soma extending point")

                    if verbose:
                        print(f"Winning Branch = {winning_branch}")

                    #3) Find the mesh and skeleton of the winning branch
                    winning_branch_mesh = divided_submeshes[winning_branch]
                    winning_branch_mesh_idx = divided_submeshes_idx[winning_branch]
                    winning_branch_sk = segment_branches[winning_branch]

                    #4) Add newly created branch to skeleton and divide the skeleton into branches (could make 2 or 3)
                    extended_skeleton_to_soma = sk.stack_skeletons([winning_branch_sk,br_new])
                    soma_extending_branches = sk.decompose_skeleton_to_branches(extended_skeleton_to_soma)

                    #5) Run Adaptive mesh correspondnece using branches and mesh
                    local_correspondnece_MP = pre.mesh_correspondence_first_pass(mesh=winning_branch_mesh,
                                                  skeleton=extended_skeleton_to_soma)

                    local_correspondence_revised = pre.correspondence_1_to_1(mesh=winning_branch_mesh,
                                                                local_correspondence=local_correspondnece_MP,
                                                                curr_limb_endpoints_must_keep=endpoints_must_keep_MP,
                                                                curr_soma_to_piece_touching_vertices=curr_soma_to_piece_touching_vertices_MP)

                    # All the things that should be revised:
                #     segment_branches, #skeleton branches
                #     divided_submeshes, divided_submeshes_idx, #mesh correspondence (mesh and indices)
                #     segment_widths_median


                    new_submeshes = [k["branch_mesh"] for k in local_correspondence_revised.values()]
                    new_submeshes_idx = [winning_branch_mesh_idx[k["branch_face_idx"]] for k in local_correspondence_revised.values()]
                    new_skeletal_branches = [k["branch_skeleton"] for k in local_correspondence_revised.values()]

                    #calculate the new width
                    ray_inter = tu.ray_pyembree.RayMeshIntersector(limb_mesh_mparty)
                    new_widths = []
                    for new_s_idx in new_submeshes_idx:
                        curr_ray_distance = tu.ray_trace_distance(mesh=limb_mesh_mparty, 
                                            face_inds=new_s_idx,
                                           ray_inter=ray_inter)
                        new_widths.append(np.median(curr_ray_distance[curr_ray_distance!=0]))


                    #6) Remove the original branch and mesh correspondence and replace with the multiples
                #     new_segment_branches = deepcopy(segment_branches)
                #     new_divided_submeshes = deepcopy(divided_submeshes)
                #     new_divided_submeshes_idx = deepcopy(divided_submeshes_idx)
                #     new_segment_widths_median = deepcopy(segment_widths_median)

                    segment_branches = np.delete(segment_branches,winning_branch)
                    segment_branches = np.append(segment_branches,new_skeletal_branches,axis=0)

                    divided_submeshes = np.delete(divided_submeshes,winning_branch)
                    divided_submeshes = np.append(divided_submeshes,new_submeshes,axis=0)

                    divided_submeshes_idx = np.delete(divided_submeshes_idx,winning_branch)
                    divided_submeshes_idx = np.append(divided_submeshes_idx,new_submeshes_idx,axis=0)

                    segment_widths_median = np.delete(segment_widths_median,winning_branch)
                    segment_widths_median = np.append(segment_widths_median,new_widths,axis=0)




            if no_soma_extension_add:
                print("No soma extending branch was added for this sublimb even though it had a soma border (means they already existed)")

            if print_fusion_steps:
                print(f"MP (because soma touching verts) soma extension add: {time.time() - fusion_time }")
                fusion_time = time.time()
            
        #building the limb correspondence
        limb_correspondence_MP[sublimb_idx] = dict()

        for zz,b_sk in enumerate(segment_branches):
            limb_correspondence_MP[sublimb_idx][zz] = dict(
                branch_skeleton = b_sk,
                width_from_skeleton = segment_widths_median[zz],
                branch_mesh = divided_submeshes[zz],
                branch_face_idx = divided_submeshes_idx[zz]
                )
            
            
            
        
        
    # ------------------------------------- Part C: Will make sure the correspondences can all be stitched together --------------- #

    # Only want to perform this step if both MP and MAP pieces
    if len(limb_correspondence_MAP)>0 and len(limb_correspondence_MP)>0:


        # ---- Doing the mesh connectivity ---------#
        sublimb_meshes_MP = []
        sublimb_skeletons_MP = []

        for sublimb_key,sublimb_v in limb_correspondence_MP.items():
            sublimb_meshes_MP.append(tu.combine_meshes([branch_v["branch_mesh"] for branch_v in sublimb_v.values()]))
            sublimb_skeletons_MP.append(sk.stack_skeletons([branch_v["branch_skeleton"] for branch_v in sublimb_v.values()]))

        sublimb_meshes_MAP = []
        sublimb_skeletons_MAP = []

        for sublimb_key,sublimb_v in limb_correspondence_MAP.items():
            sublimb_meshes_MAP.append(tu.combine_meshes([branch_v["branch_mesh"] for branch_v in sublimb_v.values()]))
            sublimb_skeletons_MAP.append(sk.stack_skeletons([branch_v["branch_skeleton"] for branch_v in sublimb_v.values()]))



        mesh_conn,mesh_conn_vertex_groups = tu.mesh_list_connectivity(meshes = sublimb_meshes_MP + sublimb_meshes_MAP,
                                            main_mesh = limb_mesh_mparty,
                                            min_common_vertices=4,
                                            return_vertex_connection_groups=True,
                                            print_flag = False)

        #check that every MAP piece mapped to a MP piece
        if np.any(mesh_conn[:,0]>=len(sublimb_meshes_MP)) or np.any(mesh_conn[:,1]<len(sublimb_meshes_MP)):
            raise Exception(f"Not all MP pieces mapped to MAP pieces \n Number of MP pieces = {len(sublimb_meshes_MP)} \n connectivity: {mesh_conn}")


        #check that the mapping should create only one connected component
        G = nx.from_edgelist(mesh_conn)
        connect_comp = list(nx.connected_components(G))
        if len(connect_comp)>1:
            raise Exception(f"Mesh connectivity was not one component, instead it was ({len(connect_comp)}): {connect_comp} ")

        #adjust the connection indices for MP and MAP indices
        mesh_conn_adjusted = np.vstack([mesh_conn[:,0],mesh_conn[:,1]-len(sublimb_meshes_MP)]).T






        """
        Pseudocode:
        For each connection edge:
            For each vertex connection group:
                1) Get the endpoint vertices of the MP skeleton
                2) Find the closest endpoint vertex to the vertex connection group (this is MP stitch point)
                3) Find the closest skeletal point on MAP pairing (MAP stitch) 
                4) Find the branches that have that MAP stitch point:
                5A) If the number of branches corresponding to stitch point is multipled
                    --> then we are stitching at a branching oint
                    i) Just add the skeletal segment from MP_stitch to MAP stitch to the MP skeletal segment
                    ii) 

        """


        for (MP_idx,MAP_idx),vert_groups in zip(mesh_conn_adjusted,mesh_conn_vertex_groups):
            print(f"\n---- Working on {(MP_idx,MAP_idx)} connection-----")

            """
            This old way of getting the endpoints was not good because could possibly just need
            a stitching done between original branch junction

            skeleton_MP_graph = sk.convert_skeleton_to_graph(curr_skeleton_MP)
            endpoint_nodes = xu.get_nodes_of_degree_k(skeleton_MP_graph,1)
            endpoint_nodes_coordinates = xu.get_node_attributes(skeleton_MP_graph,node_list=endpoint_nodes)
            """

            for v_idx,v_g in enumerate(vert_groups):
                print(f"    ---- Working on vertex group {v_idx}-----")


                #getting the skeletons that should be stitched
                curr_skeleton_MP = sk.stack_skeletons([branch_v["branch_skeleton"] for branch_v in limb_correspondence_MP[MP_idx].values()])
                curr_skeleton_MAP = sk.stack_skeletons([branch_v["branch_skeleton"] for branch_v in limb_correspondence_MAP[MAP_idx].values()])

                #1) Get the endpoint vertices of the MP skeleton (needs to be inside loop because limb correspondence will change)
                curr_MP_branch_skeletons = [k["branch_skeleton"] for k in limb_correspondence_MP[MP_idx].values()]
                endpoint_nodes_coordinates = np.array([sk.find_branch_endpoints(k) for k in curr_MP_branch_skeletons])
                endpoint_nodes_coordinates = np.unique(endpoint_nodes_coordinates.reshape(-1,3),axis=0)

                #2) Find the closest endpoint vertex to the vertex connection group (this is MP stitch point)
                av_vert = np.mean(v_g,axis=0)
                winning_vertex = endpoint_nodes_coordinates[np.argmin(np.linalg.norm(endpoint_nodes_coordinates-av_vert,axis=1))]
                print(f"winning_vertex = {winning_vertex}")


                #2b) Find the branch points where the winning vertex is located
                MP_branches_with_stitch_point = sk.find_branch_skeleton_with_specific_coordinate(
                    divded_skeleton=curr_MP_branch_skeletons,
                    current_coordinate = winning_vertex
                )
                print(f"MP_branches_with_stitch_point = {MP_branches_with_stitch_point}")


                #3) Find the closest skeletal point on MAP pairing (MAP stitch)
                MAP_skeleton_coords = np.unique(curr_skeleton_MAP.reshape(-1,3),axis=0)
                MAP_stitch_point = MAP_skeleton_coords[np.argmin(np.linalg.norm(MAP_skeleton_coords-winning_vertex,axis=1))]


                #3b) Consider if the stitch point is close enough to end or branch node in skeleton:
                # and if so then reassign
                if move_MAP_stitch_to_end_or_branch:
                    MAP_stitch_point_new,change_status = sk.move_point_to_nearest_branch_end_point_within_threshold(
                                                            graph=curr_skeleton_MAP,
                                                            coordinate=MAP_stitch_point,
                                                            distance_to_move_point_threshold = distance_to_move_point_threshold,
                                                            verbose=True

                                                            )
                    MAP_stitch_point=MAP_stitch_point_new


                #4) Find the branches that have that MAP stitch point:
                curr_MAP_branch_skeletons = [k["branch_skeleton"] for k in limb_correspondence_MAP[MAP_idx].values()]

                MAP_branches_with_stitch_point = sk.find_branch_skeleton_with_specific_coordinate(
                    divded_skeleton=curr_MAP_branch_skeletons,
                    current_coordinate = MAP_stitch_point
                )


                MAP_stitch_point_on_end_or_branch = False
                if len(MAP_branches_with_stitch_point)>1:
                    MAP_stitch_point_on_end_or_branch = True
                elif len(MAP_branches_with_stitch_point)==1:
                    if len(nu.matching_rows(sk.find_branch_endpoints(curr_MAP_branch_skeletons[MAP_branches_with_stitch_point[0]]),
                                            MAP_stitch_point))>0:
                        MAP_stitch_point_on_end_or_branch=True
                else:
                    raise Exception("No matching MAP values")


                print(f"MAP_branches_with_stitch_point = {MAP_branches_with_stitch_point}")
                print(f"MAP_stitch_point_on_end_or_branch = {MAP_stitch_point_on_end_or_branch}")


                # ------------------------- This part does the stitching -------------------- #


                """
                Pseudocode:
                1) For all MP branches
                    a) Get neighbor coordinates to MP stitch points
                    b) Delete the MP Stitch points on each 
                    c) Add skeleton segment from neighbor to MAP stitch point
                2) Get skeletons and meshes from MP and MAP pieces
                3) Run mesh correspondence to get new meshes and mesh_idx and widths
                4a) If MAP_stitch_point_on_end_or_branch is False
                - Delete the old MAP branch parts and replace with new MAP ones
                4b) Revise the meshes,  mesh_idx, and widths of the MAP pieces
                5) Revise the meshes,  mesh_idx, and widths of the MP pieces


                """

                curr_MP_sk = []
                for b_idx in MP_branches_with_stitch_point:

                    #a) Get neighbor coordinates to MP stitch points
                    MP_stitch_branch_graph = sk.convert_skeleton_to_graph(curr_MP_branch_skeletons[b_idx])
                    stitch_node = xu.get_nodes_with_attributes_dict(MP_stitch_branch_graph,dict(coordinates=winning_vertex))[0]
                    stitch_neighbors = xu.get_neighbors(MP_stitch_branch_graph,stitch_node)

                    if len(stitch_neighbors) != 1:
                        raise Exception("Not just one neighbor for stitch point of MP branch")
                    keep_neighbor = stitch_neighbors[0]  
                    keep_neighbor_coordinates = xu.get_node_attributes(MP_stitch_branch_graph,node_list=[keep_neighbor])[0]

                    #b) Delete the MP Stitch points on each 
                    MP_stitch_branch_graph.remove_node(stitch_node)

                    #c) Add skeleton segment from neighbor to MAP stitch point
                    new_node_name = np.max(MP_stitch_branch_graph.nodes())+1

                    MP_stitch_branch_graph.add_nodes_from([(int(new_node_name),{"coordinates":MAP_stitch_point})])
                    MP_stitch_branch_graph.add_weighted_edges_from([(keep_neighbor,new_node_name,np.linalg.norm(MAP_stitch_point - keep_neighbor_coordinates))])

                    new_MP_skeleton = sk.convert_graph_to_skeleton(MP_stitch_branch_graph)
                    curr_MP_sk.append(new_MP_skeleton)



                #2) Get skeletons and meshes from MP and MAP pieces
                curr_MAP_sk = [limb_correspondence_MAP[MAP_idx][k]["branch_skeleton"] for k in MAP_branches_with_stitch_point]

                #2.1) Going to break up the MAP skeleton if need be
                """
                Pseudocode:
                a) check to see if it needs to be broken up
                If it does:
                b) Convert the skeleton into a graph
                c) Find the node of the MAP stitch point (where need to do the breaking)
                d) Find the degree one nodes
                e) For each degree one node:
                - Find shortest path from stitch node to end node
                - get a subgraph from that path
                - convert graph to a skeleton and save as new skeletons

                """
                #a) check to see if it needs to be broken up
                if not MAP_stitch_point_on_end_or_branch:
                    if len(curr_MAP_sk) > 1:
                        raise Exception(f"There was more than one skeleton for MAP skeletons even though MAP_stitch_point_on_end_or_branch = {MAP_stitch_point_on_end_or_branch}")

                    curr_MAP_sk_new = []
                    #b) Convert the skeleton into a graph
                    curr_MAP_sk_graph = sk.convert_skeleton_to_graph(curr_MAP_sk[0])
                    #c) Find the node of the MAP stitch point (where need to do the breaking)
                    MP_stitch_node = xu.get_nodes_with_attributes_dict(curr_MAP_sk_graph,dict(coordinates=MAP_stitch_point))[0]
                    #d) Find the degree one nodes
                    curr_end_nodes_for_break = xu.get_nodes_of_degree_k(curr_MAP_sk_graph,1)

                    #e) For each degree one node:
                    for e_n in curr_end_nodes_for_break:
                        #- Find shortest path from stitch node to end node
                        stitch_to_end_path = nx.shortest_path(curr_MAP_sk_graph,MP_stitch_node,e_n)
                        #- get a subgraph from that path
                        stitch_to_end_path_graph = curr_MAP_sk_graph.subgraph(stitch_to_end_path)
                        #- convert graph to a skeleton and save as new skeletons
                        new_sk = sk.convert_graph_to_skeleton(stitch_to_end_path_graph)
                        curr_MAP_sk_new.append(new_sk)
                    curr_MAP_sk = curr_MAP_sk_new

                curr_MAP_meshes_idx = [limb_correspondence_MAP[MAP_idx][k]["branch_face_idx"] for k in MAP_branches_with_stitch_point]

                curr_MP_sk
                curr_MP_meshes_idx = [limb_correspondence_MP[MP_idx][k]["branch_face_idx"] for k in MP_branches_with_stitch_point]

                stitching_mesh_idx = np.concatenate(curr_MAP_meshes_idx + curr_MP_meshes_idx)
                stitching_mesh = limb_mesh_mparty.submesh([stitching_mesh_idx],append=True,repair=True)
                stitching_skeleton_branches = curr_MAP_sk + curr_MP_sk
                """

                ****** NEED TO GET THE RIGHT MESH TO RUN HE IDX ON SO GETS A GOOD MESH (CAN'T BE LIMB_MESH_MPARTY)
                BUT MUST BE THE ORIGINAL MAP MESH

                mesh_pieces_for_MAP
                sublimb_meshes_MP

                mesh_pieces_for_MAP_face_idx
                sublimb_meshes_MP_face_idx

                stitching_mesh = tu.combine_meshes(curr_MAP_meshes + curr_MP_meshes)
                stitching_skeleton_branches = curr_MAP_sk + curr_MP_sk

                """



                #3) Run mesh correspondence to get new meshes and mesh_idx and widths
                local_correspondnece_stitch = pre.mesh_correspondence_first_pass(mesh=stitching_mesh,
                                              skeleton_branches=stitching_skeleton_branches)

                local_correspondence_stitch_revised = pre.correspondence_1_to_1(mesh=stitching_mesh,
                                                            local_correspondence=local_correspondnece_stitch,
                                                            curr_limb_endpoints_must_keep=None,
                                                            curr_soma_to_piece_touching_vertices=None)

                #Need to readjust the mesh correspondence idx
                for k,v in local_correspondence_stitch_revised.items():
                    local_correspondence_stitch_revised[k]["branch_face_idx"] = stitching_mesh_idx[local_correspondence_stitch_revised[k]["branch_face_idx"]]

                #4a) If MAP_stitch_point_on_end_or_branch is False
                #- Delete the old MAP branch parts and replace with new MAP ones
                if not MAP_stitch_point_on_end_or_branch:
                    print("Deleting branches from dictionary")
                    del limb_correspondence_MAP[MAP_idx][MAP_branches_with_stitch_point[0]]
                    #adding the two new branches created from the stitching
                    limb_correspondence_MAP[MAP_idx][MAP_branches_with_stitch_point[0]] = local_correspondence_stitch_revised[0]
                    limb_correspondence_MAP[MAP_idx][np.max(list(limb_correspondence_MAP[MAP_idx].keys()))+1] = local_correspondence_stitch_revised[1]

                    #have to reorder the keys
                    limb_correspondence_MAP[MAP_idx] = dict([(k,limb_correspondence_MAP[MAP_idx][k]) for k in np.sort(list(limb_correspondence_MAP[MAP_idx].keys()))])

                else: #4b) Revise the meshes,  mesh_idx, and widths of the MAP pieces if weren't broken up
                    for j,curr_MAP_idx_fixed in enumerate(MAP_branches_with_stitch_point):
                        limb_correspondence_MAP[MAP_idx][curr_MAP_idx_fixed] = local_correspondence_stitch_revised[j]

                #5) Revise the meshes,  mesh_idx, and widths of the MP pieces
                for j,curr_MP_idx_fixed in enumerate(MP_branches_with_stitch_point):
                    limb_correspondence_MP[MP_idx][curr_MP_idx_fixed] = local_correspondence_stitch_revised[j+len(curr_MAP_sk)]


                print(f" Finished with {(MP_idx,MAP_idx)} vertex group {v_idx} \n\n\n")


    else:
        print("There were not both MAP and MP pieces so skipping the stitch resolving phase")
    
    print(f"Time for decomp of Limb {curr_limb_idx} = {time.time() - curr_limb_time}")
    # ------------- Saving the MAP and MP Decompositions ---------------- #
    proper_limb_mesh_correspondence_MAP[curr_limb_idx] = limb_correspondence_MAP
    proper_limb_mesh_correspondence_MP[curr_limb_idx] = limb_correspondence_MP
    

print(f"Total time for all limb decomps = {time.time() - proper_time}")

  0%|          | 0/1 [00:00<?, ?it/s]



----- Working on Proper Limb #5 ---------
Time for preparing soma vertices and root: 0.0001614093780517578
smooth_neighborhood = 1


100%|██████████| 1/1 [00:00<00:00, 10.84it/s]

Time for 1st pass MP skeletonization: 0.31755781173706055
branches_touching_root = [0]
empty_indices % = 0.0
 conflict_indices % = 0.0



AFTER face_lookup_resolved_test


...

all graph is one component!

Total time for skeleton stitching = 0.05731606483459473
The process was using a temp folder
    Total time for skeletonizing branch: 20.093221187591553
skipping soma 0 because closest skeleton node was already end node
    Total time for Fixing Skeleton Soma Endpoint Extension : 1.2890989780426025
filter_end_node_length = 4001
Using Distance measure skeletal_distance
Going to ignore certain endnodes that are 5000 nm close to soma border vertices
Number of end_nodes BEFORE filtering = 21
Using an already specified end node: 99 with index 1checking was correct node end_nodes[index] = 99
May Eliminate end_node 1: 99 because path_len to soma border was 890.6045062121159
single_node_to_eliminate = 1
all_single_nodes_to_eliminate = [1, 1]



********COMBINING CLOSE SKELETON NODES WITHIN 700 DISTANCE**********
Found 0 valid paths to replace
valid_paths = []
valid_path_lengths = []
No valid paths found so just returning the original
skeletonize_and_clean_connected_branch_CGAL: 22.076948881149292
Working on limb correspondence for #0 MAP piece



Total time for decomposition = 4.91597580909729
mesh_correspondence_first_pass: 4.916015386581421
Limb decomposed into 1 branches
divided_skeleton_graph_recovered = (488, 2, 3) and 
current_mesh_data[0]['branch_skeletons_cleaned'].shape = (488, 2, 3)

Number of connected components in deocmposed recovered graph = 1
Number of connected components in cleaned skeleton graph= 1
The downsampled branches number of connected components = 1
Empty submeshes = []


--- Working on 1-to-1 correspondence-----
max(original_labels),len(original_labels) = (0, 1)
empty_indices % = 0.011198614834673815
 conflict_indices % = 0.0



AFTER face_lookup_resolved_test
Took 0 iterations to expand the label back



Total time for MAP sublimb #0 mesh processing = 27.7180438041687
correspondence_1_to_1: 0.7225198745727539
Total time for MAP sublimb processing 27.71817970275879
***************Arrived at MESH PIECES FOR MAP AND mp***************
There were not both MAP and MP pieces so skipping the stitch resolving phase
Time for decomp of Limb 5 = 28.611798763275146
Total time for all limb decomps = 28.614373207092285


In [141]:
mesh_conn

array([], dtype=float64)

# Visualizing the Results before stitching

In [143]:
MAP_skeletons = []
MAP_meshes = []

for proper_idx,limb_correspondence_MAP in proper_limb_mesh_correspondence_MAP.items():
    for MAP_sublimb_idx,v in limb_correspondence_MAP.items():
        for MAP_branch_idx,v2 in v.items():
            MAP_skeletons.append(v2["branch_skeleton"])
            MAP_meshes.append(v2["branch_mesh"])

MP_skeletons = []
MP_meshes = []

for proper_idx,limb_correspondence_MP in proper_limb_mesh_correspondence_MP.items():
    for MP_sublimb_idx,v in limb_correspondence_MP.items():
        for MP_branch_idx,v2 in v.items():
            MP_skeletons.append(v2["branch_skeleton"])
            MP_meshes.append(v2["branch_mesh"])

nviz.plot_objects(main_mesh=tu.combine_meshes(soma_touching_mesh_data[z]["soma_meshes"]),
                  main_mesh_color="red",
    meshes=MP_meshes + MAP_meshes,
                  meshes_colors="random",
                 skeletons=MP_skeletons + MAP_skeletons,
                 skeletons_colors="random",
                #html_path="cloud_neuron_decomp_test.html",
#show_at_end=False
                 )

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


# checking that a branch has all one connected component skeleton

In [147]:
MAP_skeletons = []
MAP_meshes = []

check_idx = 0

limb_correspondence_MAP = proper_limb_mesh_correspondence_MAP[check_idx]
for MAP_sublimb_idx,v in limb_correspondence_MAP.items():
    for MAP_branch_idx,v2 in v.items():
        MAP_skeletons.append(v2["branch_skeleton"])
        MAP_meshes.append(v2["branch_mesh"])

MP_skeletons = []
MP_meshes = []

limb_correspondence_MP = proper_limb_mesh_correspondence_MP[check_idx]
for MP_sublimb_idx,v in limb_correspondence_MP.items():
    for MP_branch_idx,v2 in v.items():
        MP_skeletons.append(v2["branch_skeleton"])
        MP_meshes.append(v2["branch_mesh"])
        


In [148]:
total_limb_sk = sk.stack_skeletons(MAP_skeletons+MP_skeletons)
nx.number_connected_components(sk.convert_skeleton_to_graph(total_limb_sk))

1

# ---------- Resuming What will go in the function ----------

# Doing the Stitching

In [17]:
# nviz.plot_objects(main_mesh=sublimb_meshes_MAP[0],
#     meshes=sublimb_meshes_MP,
#                  meshes_colors=["red","yellow","pink","black","brown","orange"])

In [126]:
# Testing if we can get mesh connectivity
curr_limb_idx = 1

limb_mesh_mparty = current_mesh_data[0]["branch_meshes"][curr_limb_idx]

limb_correspondence_MP = copy.deepcopy(proper_limb_mesh_correspondence_MP[curr_limb_idx])
limb_correspondence_MAP = copy.deepcopy(proper_limb_mesh_correspondence_MAP[curr_limb_idx])

In [127]:
# #MP_branches = [limb_correspondence_MP[0][k]["branch_mesh"] for k in MP_branches_with_stitch_point]
# #MAP_branches = [limb_correspondence_MAP[MAP_idx][k]["branch_mesh"] for k in MAP_branches_with_stitch_point]
# MAP_branches_whole = [v["branch_mesh"] for k,v in limb_correspondence_MAP[MAP_idx].items()]

# nviz.plot_objects(
#     #main_mesh=mesh_pieces_for_MAP[0],
#     meshes=MAP_branches_whole,
#                 #skeletons=curr_MAP_sk,
#                  scatters=[MAP_stitch_point],
#                  scatter_size=0.3)


In [129]:
# import neuron_visualizations as nviz
# nviz.plot_objects(meshes=sublimb_meshes_MP+sublimb_meshes_MAP,
#                  meshes_colors="random")

# trying out the mesh connectivity

array([[0, 0],
       [1, 0],
       [2, 0],
       [3, 0],
       [4, 0],
       [5, 0]])

# Finding the MP skeletal endpoints and the MP skeletal points to stitch to 

In [131]:

# Testing if we can get mesh connectivity
sublimb_meshes_MP = []
sublimb_skeletons_MP = []

for sublimb_key,sublimb_v in limb_correspondence_MP.items():
    sublimb_meshes_MP.append(tu.combine_meshes([branch_v["branch_mesh"] for branch_v in sublimb_v.values()]))
    sublimb_skeletons_MP.append(sk.stack_skeletons([branch_v["branch_skeleton"] for branch_v in sublimb_v.values()]))

sublimb_meshes_MAP = []
sublimb_skeletons_MAP = []

for sublimb_key,sublimb_v in limb_correspondence_MAP.items():
    sublimb_meshes_MAP.append(tu.combine_meshes([branch_v["branch_mesh"] for branch_v in sublimb_v.values()]))
    sublimb_skeletons_MAP.append(sk.stack_skeletons([branch_v["branch_skeleton"] for branch_v in sublimb_v.values()]))
        
#proper_limb_mesh_correspondence_MAP

mesh_conn,mesh_conn_vertex_groups = tu.mesh_list_connectivity(meshes = sublimb_meshes_MP + sublimb_meshes_MAP,
                                    main_mesh = limb_mesh_mparty,
                                    min_common_vertices=4,
                                    return_vertex_connection_groups=True,
                                    print_flag = False)

#check that every MAP piece mapped to a MP piece
if np.any(mesh_conn[:,0]>=len(sublimb_meshes_MP)) or np.any(mesh_conn[:,1]<len(sublimb_meshes_MP)):
    raise Exception(f"Not all MP pieces mapped to MAP pieces \n Number of MP pieces = {len(sublimb_meshes_MP)} \n connectivity: {mesh_conn}")


#check that the mapping should create only one connected component
G = nx.from_edgelist(mesh_conn)
connect_comp = list(nx.connected_components(G))
if len(connect_comp)>1:
    raise Exception(f"Mesh connectivity was not one component, instead it was ({len(connect_comp)}): {connect_comp} ")

#adjust the connection indices for MP and MAP indices
mesh_conn_adjusted = np.vstack([mesh_conn[:,0],mesh_conn[:,1]-len(sublimb_meshes_MP)]).T

"""
Pseudocode:
For each connection edge:
    For each vertex connection group:
        1) Get the endpoint vertices of the MP skeleton
        2) Find the closest endpoint vertex to the vertex connection group (this is MP stitch point)
        3) Find the closest skeletal point on MAP pairing (MAP stitch) 
        4) Find the branches that have that MAP stitch point:
        5A) If the number of branches corresponding to stitch point is multipled
            --> then we are stitching at a branching oint
            i) Just add the skeletal segment from MP_stitch to MAP stitch to the MP skeletal segment
            ii) 

"""


move_MAP_stitch_to_end_or_branch = True
distance_to_move_point_threshold=1000


for (MP_idx,MAP_idx),vert_groups in zip(mesh_conn_adjusted,mesh_conn_vertex_groups):
    print(f"\n---- Working on {(MP_idx,MAP_idx)} connection-----")
    
    """
    This old way of getting the endpoints was not good because could possibly just need
    a stitching done between original branch junction
    
    skeleton_MP_graph = sk.convert_skeleton_to_graph(curr_skeleton_MP)
    endpoint_nodes = xu.get_nodes_of_degree_k(skeleton_MP_graph,1)
    endpoint_nodes_coordinates = xu.get_node_attributes(skeleton_MP_graph,node_list=endpoint_nodes)
    """

    for v_idx,v_g in enumerate(vert_groups):
        print(f"    ---- Working on vertex group {v_idx}-----")
    
        
        #getting the skeletons that should be stitched
        curr_skeleton_MP = sk.stack_skeletons([branch_v["branch_skeleton"] for branch_v in limb_correspondence_MP[MP_idx].values()])
        curr_skeleton_MAP = sk.stack_skeletons([branch_v["branch_skeleton"] for branch_v in limb_correspondence_MAP[MAP_idx].values()])
        
        #1) Get the endpoint vertices of the MP skeleton (needs to be inside loop because limb correspondence will change)
        curr_MP_branch_skeletons = [k["branch_skeleton"] for k in limb_correspondence_MP[MP_idx].values()]
        endpoint_nodes_coordinates = np.array([sk.find_branch_endpoints(k) for k in curr_MP_branch_skeletons])
        endpoint_nodes_coordinates = np.unique(endpoint_nodes_coordinates.reshape(-1,3),axis=0)
        
        #2) Find the closest endpoint vertex to the vertex connection group (this is MP stitch point)
        av_vert = np.mean(v_g,axis=0)
        winning_vertex = endpoint_nodes_coordinates[np.argmin(np.linalg.norm(endpoint_nodes_coordinates-av_vert,axis=1))]
        print(f"winning_vertex = {winning_vertex}")
        
        
        #2b) Find the branch points where the winning vertex is located
        MP_branches_with_stitch_point = sk.find_branch_skeleton_with_specific_coordinate(
            divded_skeleton=curr_MP_branch_skeletons,
            current_coordinate = winning_vertex
        )
        print(f"MP_branches_with_stitch_point = {MP_branches_with_stitch_point}")
        
        
        #3) Find the closest skeletal point on MAP pairing (MAP stitch)
        MAP_skeleton_coords = np.unique(curr_skeleton_MAP.reshape(-1,3),axis=0)
        MAP_stitch_point = MAP_skeleton_coords[np.argmin(np.linalg.norm(MAP_skeleton_coords-winning_vertex,axis=1))]
        
        
        #3b) Consider if the stitch point is close enough to end or branch node in skeleton:
        # and if so then reassign
        if move_MAP_stitch_to_end_or_branch:
            MAP_stitch_point_new,change_status = sk.move_point_to_nearest_branch_end_point_within_threshold(
                                                    graph=curr_skeleton_MAP,
                                                    coordinate=MAP_stitch_point,
                                                    distance_to_move_point_threshold = distance_to_move_point_threshold,
                                                    verbose=True

                                                    )
            MAP_stitch_point=MAP_stitch_point_new
        
        
        #4) Find the branches that have that MAP stitch point:
        curr_MAP_branch_skeletons = [k["branch_skeleton"] for k in limb_correspondence_MAP[MAP_idx].values()]
        
        MAP_branches_with_stitch_point = sk.find_branch_skeleton_with_specific_coordinate(
            divded_skeleton=curr_MAP_branch_skeletons,
            current_coordinate = MAP_stitch_point
        )
        
        
        MAP_stitch_point_on_end_or_branch = False
        if len(MAP_branches_with_stitch_point)>1:
            MAP_stitch_point_on_end_or_branch = True
        elif len(MAP_branches_with_stitch_point)==1:
            if len(nu.matching_rows(sk.find_branch_endpoints(curr_MAP_branch_skeletons[MAP_branches_with_stitch_point[0]]),
                                    MAP_stitch_point))>0:
                MAP_stitch_point_on_end_or_branch=True
        else:
            raise Exception("No matching MAP values")
        
        
        print(f"MAP_branches_with_stitch_point = {MAP_branches_with_stitch_point}")
        print(f"MAP_stitch_point_on_end_or_branch = {MAP_stitch_point_on_end_or_branch}")
        
        
        # ------------------------- This part does the stitching -------------------- #
        
        
        """
        Pseudocode:
        1) For all MP branches
            a) Get neighbor coordinates to MP stitch points
            b) Delete the MP Stitch points on each 
            c) Add skeleton segment from neighbor to MAP stitch point
        2) Get skeletons and meshes from MP and MAP pieces
        3) Run mesh correspondence to get new meshes and mesh_idx and widths
        4a) If MAP_stitch_point_on_end_or_branch is False
        - Delete the old MAP branch parts and replace with new MAP ones
        4b) Revise the meshes,  mesh_idx, and widths of the MAP pieces
        5) Revise the meshes,  mesh_idx, and widths of the MP pieces


        """

        curr_MP_sk = []
        for b_idx in MP_branches_with_stitch_point:

            #a) Get neighbor coordinates to MP stitch points
            MP_stitch_branch_graph = sk.convert_skeleton_to_graph(curr_MP_branch_skeletons[b_idx])
            stitch_node = xu.get_nodes_with_attributes_dict(MP_stitch_branch_graph,dict(coordinates=winning_vertex))[0]
            stitch_neighbors = xu.get_neighbors(MP_stitch_branch_graph,stitch_node)

            if len(stitch_neighbors) != 1:
                raise Exception("Not just one neighbor for stitch point of MP branch")
            keep_neighbor = stitch_neighbors[0]  
            keep_neighbor_coordinates = xu.get_node_attributes(MP_stitch_branch_graph,node_list=[keep_neighbor])[0]

            #b) Delete the MP Stitch points on each 
            MP_stitch_branch_graph.remove_node(stitch_node)

            #c) Add skeleton segment from neighbor to MAP stitch point
            new_node_name = np.max(MP_stitch_branch_graph.nodes())+1

            MP_stitch_branch_graph.add_nodes_from([(int(new_node_name),{"coordinates":MAP_stitch_point})])
            MP_stitch_branch_graph.add_weighted_edges_from([(keep_neighbor,new_node_name,np.linalg.norm(MAP_stitch_point - keep_neighbor_coordinates))])

            new_MP_skeleton = sk.convert_graph_to_skeleton(MP_stitch_branch_graph)
            curr_MP_sk.append(new_MP_skeleton)



        #2) Get skeletons and meshes from MP and MAP pieces
        curr_MAP_sk = [limb_correspondence_MAP[MAP_idx][k]["branch_skeleton"] for k in MAP_branches_with_stitch_point]

        #2.1) Going to break up the MAP skeleton if need be
        """
        Pseudocode:
        a) check to see if it needs to be broken up
        If it does:
        b) Convert the skeleton into a graph
        c) Find the node of the MAP stitch point (where need to do the breaking)
        d) Find the degree one nodes
        e) For each degree one node:
        - Find shortest path from stitch node to end node
        - get a subgraph from that path
        - convert graph to a skeleton and save as new skeletons

        """
        #a) check to see if it needs to be broken up
        if not MAP_stitch_point_on_end_or_branch:
            if len(curr_MAP_sk) > 1:
                raise Exception(f"There was more than one skeleton for MAP skeletons even though MAP_stitch_point_on_end_or_branch = {MAP_stitch_point_on_end_or_branch}")

            curr_MAP_sk_new = []
            #b) Convert the skeleton into a graph
            curr_MAP_sk_graph = sk.convert_skeleton_to_graph(curr_MAP_sk[0])
            #c) Find the node of the MAP stitch point (where need to do the breaking)
            MP_stitch_node = xu.get_nodes_with_attributes_dict(curr_MAP_sk_graph,dict(coordinates=MAP_stitch_point))[0]
            #d) Find the degree one nodes
            curr_end_nodes_for_break = xu.get_nodes_of_degree_k(curr_MAP_sk_graph,1)

            #e) For each degree one node:
            for e_n in curr_end_nodes_for_break:
                #- Find shortest path from stitch node to end node
                stitch_to_end_path = nx.shortest_path(curr_MAP_sk_graph,MP_stitch_node,e_n)
                #- get a subgraph from that path
                stitch_to_end_path_graph = curr_MAP_sk_graph.subgraph(stitch_to_end_path)
                #- convert graph to a skeleton and save as new skeletons
                new_sk = sk.convert_graph_to_skeleton(stitch_to_end_path_graph)
                curr_MAP_sk_new.append(new_sk)
            curr_MAP_sk = curr_MAP_sk_new

        curr_MAP_meshes_idx = [limb_correspondence_MAP[MAP_idx][k]["branch_face_idx"] for k in MAP_branches_with_stitch_point]

        curr_MP_sk
        curr_MP_meshes_idx = [limb_correspondence_MP[MP_idx][k]["branch_face_idx"] for k in MP_branches_with_stitch_point]

        stitching_mesh_idx = np.concatenate(curr_MAP_meshes_idx + curr_MP_meshes_idx)
        stitching_mesh = limb_mesh_mparty.submesh([stitching_mesh_idx],append=True,repair=True)
        stitching_skeleton_branches = curr_MAP_sk + curr_MP_sk
        """

        ****** NEED TO GET THE RIGHT MESH TO RUN HE IDX ON SO GETS A GOOD MESH (CAN'T BE LIMB_MESH_MPARTY)
        BUT MUST BE THE ORIGINAL MAP MESH

        mesh_pieces_for_MAP
        sublimb_meshes_MP

        mesh_pieces_for_MAP_face_idx
        sublimb_meshes_MP_face_idx

        stitching_mesh = tu.combine_meshes(curr_MAP_meshes + curr_MP_meshes)
        stitching_skeleton_branches = curr_MAP_sk + curr_MP_sk

        """



        #3) Run mesh correspondence to get new meshes and mesh_idx and widths
        local_correspondnece_stitch = pre.mesh_correspondence_first_pass(mesh=stitching_mesh,
                                      skeleton_branches=stitching_skeleton_branches)

        local_correspondence_stitch_revised = pre.correspondence_1_to_1(mesh=stitching_mesh,
                                                    local_correspondence=local_correspondnece_stitch,
                                                    curr_limb_endpoints_must_keep=None,
                                                    curr_soma_to_piece_touching_vertices=None)

        #Need to readjust the mesh correspondence idx
        for k,v in local_correspondence_stitch_revised.items():
            local_correspondence_stitch_revised[k]["branch_face_idx"] = stitching_mesh_idx[local_correspondence_stitch_revised[k]["branch_face_idx"]]

        #4a) If MAP_stitch_point_on_end_or_branch is False
        #- Delete the old MAP branch parts and replace with new MAP ones
        if not MAP_stitch_point_on_end_or_branch:
            print("Deleting branches from dictionary")
            del limb_correspondence_MAP[MAP_idx][MAP_branches_with_stitch_point[0]]
            #adding the two new branches created from the stitching
            limb_correspondence_MAP[MAP_idx][MAP_branches_with_stitch_point[0]] = local_correspondence_stitch_revised[0]
            limb_correspondence_MAP[MAP_idx][np.max(list(limb_correspondence_MAP[MAP_idx].keys()))+1] = local_correspondence_stitch_revised[1]
            
            #have to reorder the keys
            limb_correspondence_MAP[MAP_idx] = dict([(k,limb_correspondence_MAP[MAP_idx][k]) for k in np.sort(list(limb_correspondence_MAP[MAP_idx].keys()))])
            
        else: #4b) Revise the meshes,  mesh_idx, and widths of the MAP pieces if weren't broken up
            for j,curr_MAP_idx_fixed in enumerate(MAP_branches_with_stitch_point):
                limb_correspondence_MAP[MAP_idx][curr_MAP_idx_fixed] = local_correspondence_stitch_revised[j]

        #5) Revise the meshes,  mesh_idx, and widths of the MP pieces
        for j,curr_MP_idx_fixed in enumerate(MP_branches_with_stitch_point):
            limb_correspondence_MP[MP_idx][curr_MP_idx_fixed] = local_correspondence_stitch_revised[j+len(curr_MAP_sk)]
            
            
        print(f" Finished with {(MP_idx,MAP_idx)} vertex group {v_idx} \n\n\n")
        


---- Working on (0, 0) connection-----
    ---- Working on vertex group 0-----
winning_vertex = [505650.10601563 746001.25483436 923569.77901208]
MP_branches_with_stitch_point = [1]
Current stitch point was not a branch or endpoint, shortest_path_length to one = 34491.840120420864
MAP_branches_with_stitch_point = [0]
MAP_stitch_point_on_end_or_branch = False





--- Working on 1-to-1 correspondence-----
max(original_labels),len(original_labels) = (2, 3)
empty_indices % = 0.011904184648208312
 conflict_indices % = 0.007612859428793095



AFTER face_lookup_resolved_test



Deleting branches from dictionary
 Finished with (0, 0) vertex group 0 




---- Working on (1, 0) connection-----
    ---- Working on vertex group 0-----
winning_vertex = [591315.62600457 755607.78127527 885966.08547152]
MP_branches_with_stitch_point = [0]
Current stitch point was not a branch or endpoint, shortest_path_length to one = 17742.10493205283
MAP_branches_with_stitch_point = [13]
MAP_stitch_point_on_end_or_branch = False





--- Working on 1-to-1 correspondence-----
max(original_labels),len(original_labels) = (2, 3)
empty_indices % = 0.02556463625749829
 conflict_indices % = 0.0072064092757357385


...


AFTER face_lookup_resolved_test



 Finished with (4, 0) vertex group 0 




---- Working on (5, 0) connection-----
    ---- Working on vertex group 0-----
winning_vertex = [597327.89804131 746508.67764411 884683.08259358]
MP_branches_with_stitch_point = [0]
Current stitch point was not a branch or endpoint, shortest_path_length to one = 29596.49000093336
MAP_branches_with_stitch_point = [24]
MAP_stitch_point_on_end_or_branch = False





--- Working on 1-to-1 correspondence-----
max(original_labels),len(original_labels) = (2, 3)
empty_indices % = 0.013422495910709131
 conflict_indices % = 0.009333205041855095



AFTER face_lookup_resolved_test



Deleting branches from dictionary
 Finished with (5, 0) vertex group 0 





# visualizing whole limb decomposition went well

In [135]:
MAP_skeletons = []
MAP_meshes = []


for MAP_sublimb_idx,v in limb_correspondence_MAP.items():
    for MAP_branch_idx,v2 in v.items():
        MAP_skeletons.append(v2["branch_skeleton"])
        MAP_meshes.append(v2["branch_mesh"])

MP_skeletons = []
MP_meshes = []


for MP_sublimb_idx,v in limb_correspondence_MP.items():
    for MP_branch_idx,v2 in v.items():
        MP_skeletons.append(v2["branch_skeleton"])
        MP_meshes.append(v2["branch_mesh"])

nviz.plot_objects(main_mesh=soma_touching_mesh_data[z]["soma_meshes"][0],
                  main_mesh_color="red",
    meshes=MP_meshes + MAP_meshes,
                  meshes_colors="random",
                 skeletons=MP_skeletons + MAP_skeletons,
                 skeletons_colors="random",
                #html_path="cloud_neuron_decomp_test.html",
#show_at_end=False
                 )

# Checking that stitching went well

In [32]:

branch_dicts_MAP = [limb_correspondence_MAP[MAP_idx][MAP_branches_with_stitch_point[0]],
                limb_correspondence_MAP[MAP_idx][np.max(list(limb_correspondence_MAP[MAP_idx].keys()))]]

branch_dicts_MP = []
for j,curr_MP_idx_fixed in enumerate(MP_branches_with_stitch_point):
    branch_dicts_MP.append(limb_correspondence_MP[MP_idx][curr_MP_idx_fixed])

    
nviz.plot_objects(meshes=[k["branch_mesh"] for k in branch_dicts_MAP+branch_dicts_MP],
                  meshes_colors="random",
                 skeletons=[k["branch_skeleton"] for k in branch_dicts_MAP+branch_dicts_MP],
                 skeletons_colors="random")

combined_skel = sk.stack_skeletons([k["branch_skeleton"] for k in branch_dicts_MAP+branch_dicts_MP])
nx.number_connected_components(sk.convert_skeleton_to_graph(combined_skel))

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


1